In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd # for reading .txt, .csv data
import numpy as np # from panda to np array?

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 3 # originally 20, tried to make as small as possible. Three appears to be the minimum that still gives good recosntructions
num_epochs = 20 # was 15
batch_size = 50 # original 128
learning_rate = 1e-3
beta = 1 # 1 is VAE, > 1 for bVAE, which should give less accurate reconstructions but more meaningful latents

# MNIST dataset - original, unused
dataset = torchvision.datasets.MNIST(root='../../data',
                                     train=True,
                                     transform=transforms.ToTensor(),
                                     download=True)



#New synthetic dataset, used by rest of code, generated from generate.m file
def loadfn(path):
    data = pd.read_csv(path, sep = ",", header=None)
    data = np.asarray(data)
    data = data.astype(float)
    return np.reshape(data,[28,28,1])
    

dataset = torchvision.datasets.DatasetFolder(root='../../data/synth',
                                             loader= loadfn,
                                             extensions = ['txt'],
                                             transform=transforms.ToTensor(),
                                             target_transform=None)


# Test code to make sure images load properly
#test = loadfn('../../data/synth/c1/testdat1.txt')
#print(test)
#test2 = pd.read_csv('../../data/synth/c1/testdat1.txt', sep = ",", header=None)
#print(test2)
       


# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


# VAE model - unmodified. Other architectures, including temporal versions of VAE would undoubtedly offer better performance
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=3):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    
# VAE2 - modified with an extra hidden layer
class VAE2(nn.Module):
    def __init__(self, image_size=784, h_dim=400, h_dim2 = 150, z_dim=3):
        super(VAE2, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2e= nn.Linear(h_dim, h_dim2)
        self.fc2 = nn.Linear(h_dim2, z_dim)
        self.fc3e= nn.Linear(h_dim, h_dim2)
        self.fc3 = nn.Linear(h_dim2, z_dim)
        self.fc4e= nn.Linear(z_dim, h_dim2)
        self.fc4 = nn.Linear(h_dim2,h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2e(h))
        h2 = F.relu(self.fc3e(h))
        return self.fc2(h1), self.fc3(h2)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4e(z))
        h = F.relu(self.fc4(h))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var    

model = VAE2().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.float()
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + beta*kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))
        
print(list(mu.size())) # mu, sigma are given per minibatch

# Testing - should just need to go over all inputs once, save z
with torch.no_grad():
    for i, (x, _) in enumerate(test_loader):
        x = x.float()
        x = x.to(device).view(-1, image_size)
        _, mu, log_var = model(x)
        if (i==0):
            muout = mu
            logvarout = log_var
        else:
            muout = torch.cat([muout,mu],0)
            logvarout = torch.cat([logvarout,log_var],0)
            
#print(list(muout.size())) # correctly gets full list of mu, sigma. 
muout = muout.cpu().numpy()
logvarout = logvarout.cpu().numpy()
print(np.shape(muout))
print(muout[0,:])

# Save output, which can then be used to training a RNN e.g. FORCE
muout.tofile('muout.csv',sep=',',format='%10.5f')
logvarout.tofile('logvarout.csv',sep=',',format='%10.5f')

# Save model params, so we can load trained RNN predictions
torch.save(model.state_dict(), 'VAEparams.ckpt')
# to load
#the_model = VAE().to(device)
#VAE.load_state_dict(torch.load(PATH))

D:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/20], Step [10/20], Reconst Loss: 13521.6094, KL Div: 6.1914
Epoch[1/20], Step [20/20], Reconst Loss: 7962.4883, KL Div: 62.6063
Epoch[2/20], Step [10/20], Reconst Loss: 6679.8228, KL Div: 1.6354
Epoch[2/20], Step [20/20], Reconst Loss: 6620.9077, KL Div: 0.3061
Epoch[3/20], Step [10/20], Reconst Loss: 6698.9663, KL Div: 0.2039
Epoch[3/20], Step [20/20], Reconst Loss: 6504.1895, KL Div: 0.1980
Epoch[4/20], Step [10/20], Reconst Loss: 6451.3721, KL Div: 0.1980
Epoch[4/20], Step [20/20], Reconst Loss: 6330.2559, KL Div: 0.1973
Epoch[5/20], Step [10/20], Reconst Loss: 6276.2324, KL Div: 0.2676
Epoch[5/20], Step [20/20], Reconst Loss: 6274.7720, KL Div: 0.4154
Epoch[6/20], Step [10/20], Reconst Loss: 6651.2788, KL Div: 0.8276
Epoch[6/20], Step [20/20], Reconst Loss: 6312.4946, KL Div: 1.0684
Epoch[7/20], Step [10/20], Reconst Loss: 6302.5020, KL Div: 1.9354
Epoch[7/20], Step [20/20], Reconst Loss: 6459.8213, KL Div: 3.0621
Epoch[8/20], Step [10/20], Reconst Loss: 6149.0786, KL Div: 

In [1]:
# Import saved model
# Import predicted paths (from matlab or etc)
# Output predicted images from paths

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd # for reading .txt, .csv data
import numpy as np # from panda to np array?

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=3):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    
# Try VAE2 with an extra hidden layer
class VAE2(nn.Module):
    def __init__(self, image_size=784, h_dim=400, h_dim2 = 150, z_dim=3):
        super(VAE2, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2e= nn.Linear(h_dim, h_dim2)
        self.fc2 = nn.Linear(h_dim2, z_dim)
        self.fc3e= nn.Linear(h_dim, h_dim2)
        self.fc3 = nn.Linear(h_dim2, z_dim)
        self.fc4e= nn.Linear(z_dim, h_dim2)
        self.fc4 = nn.Linear(h_dim2,h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2e(h))
        h2 = F.relu(self.fc3e(h))
        return self.fc2(h1), self.fc3(h2)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4e(z))
        h = F.relu(self.fc4(h))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var   


# Load model
model = VAE2().to(device)
model.load_state_dict(torch.load('VAEparams.ckpt'))

# Load predicted data from matlab
muin = pd.read_csv('mupred.csv', sep = ",", header=None)
muin = np.asarray(muin)
muin = muin.astype(float)
muin = torch.tensor(muin)

logvarin = pd.read_csv('logvarpred.csv', sep = ",", header=None)
logvarin = np.asarray(logvarin)
logvarin = logvarin.astype(float)
logvarin = torch.tensor(logvarin)

# z = mu + eps * std, std = torch.exp(log_var/2), eps = torch.randn_like(std)
std = torch.exp(logvarin/2)
eps = torch.randn_like(std)
zin = muin + eps * std

zin = zin.float()
zin = zin.cuda()

#out = model.decode(zin[0,0:3]).view(-1, 1, 28, 28)

zinsm = zin[0,0:150]
dim = (50,3)
print(dim)
zinsm = torch.reshape(zinsm,dim)

out = model.decode(zinsm).view(-1, 1, 28, 28) # first 50?
save_image(out, os.path.join('predicted.png'))

zinsm2 = zin[0,150:300]
zinsm2 = torch.reshape(zinsm2,dim)

out2 = model.decode(zinsm2).view(-1, 1, 28, 28) # next 50
save_image(out2, os.path.join('predicted2.png'))

zinsm3= zin[0,300:450]
zinsm3 = torch.reshape(zinsm3,dim)

out3 = model.decode(zinsm3).view(-1, 1, 28, 28) # next 50
save_image(out3, os.path.join('predicted3.png'))
        


(50, 3)


In [6]:
# Import saved model
# Output images made from varying one latent variable at a time
# ALlows seeing what each one corresponds to

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd # for reading .txt, .csv data
import numpy as np # from panda to np array?

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=3):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    
# Try VAE2 with an extra hidden layer
class VAE2(nn.Module):
    def __init__(self, image_size=784, h_dim=400, h_dim2 = 150, z_dim=3):
        super(VAE2, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2e= nn.Linear(h_dim, h_dim2)
        self.fc2 = nn.Linear(h_dim2, z_dim)
        self.fc3e= nn.Linear(h_dim, h_dim2)
        self.fc3 = nn.Linear(h_dim2, z_dim)
        self.fc4e= nn.Linear(z_dim, h_dim2)
        self.fc4 = nn.Linear(h_dim2,h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2e(h))
        h2 = F.relu(self.fc3e(h))
        return self.fc2(h1), self.fc3(h2)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4e(z))
        h = F.relu(self.fc4(h))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var   


# Load model
model = VAE2().to(device)
model.load_state_dict(torch.load('VAEparams.ckpt'))

# Load perturbed latents, to see what each one controls
vary = np.linspace(-2, 2, num=50)
#vary = np.transpose(vary)
zeros = np.zeros((50,))
zin1 = np.vstack((vary,zeros,zeros))
zin2 = np.vstack((zeros,vary,zeros))
zin3 = np.vstack((zeros,zeros,vary))

zin1 = np.transpose(zin1)
zin1 = np.asarray(zin1)
zin1 = zin1.astype(float)
zin1 = torch.tensor(zin1)
zin1 = zin1.float()
zin1 = zin1.cuda()

zin2 = np.transpose(zin2)
zin2 = np.asarray(zin2)
zin2 = zin2.astype(float)
zin2 = torch.tensor(zin2)
zin2 = zin2.float()
zin2 = zin2.cuda()

zin3 = np.transpose(zin3)
zin3 = np.asarray(zin3)
zin3 = zin3.astype(float)
zin3 = torch.tensor(zin3)
zin3 = zin3.float()
zin3 = zin3.cuda()

#out = model.decode(zin[0,0:3]).view(-1, 1, 28, 28)

dim = (50,3)
zin1sm = torch.reshape(zin1,dim)
zin2sm = torch.reshape(zin2,dim)
zin3sm = torch.reshape(zin3,dim)

out1 = model.decode(zin1sm).view(-1, 1, 28, 28) # first 50?
save_image(out1, os.path.join('varyl1.png'))

out2 = model.decode(zin2sm).view(-1, 1, 28, 28) # first 50?
save_image(out2, os.path.join('varyl2.png'))

out3 = model.decode(zin3sm).view(-1, 1, 28, 28) # first 50?
save_image(out3, os.path.join('varyl3.png'))
        

In [20]:
# Similar to above - calculate Riemann Metrics at each point
# norm(Riemann metric * derivative vector) gives cost of moving in derivative vector direction
# Try starting from all 0s, calcaulte Reimann metric, find best / worst directions to move in

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd # for reading .txt, .csv data
import numpy as np # from panda to np array?

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=3):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var
    
# Try VAE2 with an extra hidden layer
class VAE2(nn.Module):
    def __init__(self, image_size=784, h_dim=400, h_dim2 = 150, z_dim=3):
        super(VAE2, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2e= nn.Linear(h_dim, h_dim2)
        self.fc2 = nn.Linear(h_dim2, z_dim)
        self.fc3e= nn.Linear(h_dim, h_dim2)
        self.fc3 = nn.Linear(h_dim2, z_dim)
        self.fc4e= nn.Linear(z_dim, h_dim2)
        self.fc4 = nn.Linear(h_dim2,h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        h1 = F.relu(self.fc2e(h))
        h2 = F.relu(self.fc3e(h))
        return self.fc2(h1), self.fc3(h2)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4e(z))
        h = F.relu(self.fc4(h))
        return torch.sigmoid(self.fc5(h))
        #return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var   


# Load model
model = VAE2().to(device)
model.load_state_dict(torch.load('VAEparams.ckpt'))

vary = np.linspace(-.01, .01,num=2)
#vary = np.transpose(vary)
zeros = np.zeros((2,))

zin1 = np.vstack((vary,zeros,zeros))
zin2 = np.vstack((zeros,vary,zeros))
zin3 = np.vstack((zeros,zeros,vary))

zin1 = np.transpose(zin1)
zin1 = np.asarray(zin1)
zin1 = zin1.astype(float)
zin1 = torch.tensor(zin1)
zin1 = zin1.float()
zin1 = zin1.cuda()

zin2 = np.transpose(zin2)
zin2 = np.asarray(zin2)
zin2 = zin2.astype(float)
zin2 = torch.tensor(zin2)
zin2 = zin2.float()
zin2 = zin2.cuda()

zin3 = np.transpose(zin3)
zin3 = np.asarray(zin3)
zin3 = zin3.astype(float)
zin3 = torch.tensor(zin3)
zin3 = zin3.float()
zin3 = zin3.cuda()

dim = (2,3)
zin1sm = torch.reshape(zin1,dim)
zin2sm = torch.reshape(zin2,dim)
zin3sm = torch.reshape(zin3,dim)

with torch.no_grad():

    out1 = model.decode(zin1sm).view(-1, 1, 28, 28) 

    out2 = model.decode(zin2sm).view(-1, 1, 28, 28)

    out3 = model.decode(zin3sm).view(-1, 1, 28, 28) 

# outputs now 2x1x28x28
# take difference

d1 = out1[1,0,:,:]-out1[0,0,:,:]
d1 = d1/.02

d2 = out2[1,0,:,:]-out2[0,0,:,:]
d2 = d2/.02

d3 = out3[1,0,:,:]-out3[0,0,:,:]
d3 = d3/.02

print(np.linalg.norm(d1)) 
print(np.linalg.norm(d2))
print(np.linalg.norm(d3))

# says direction 2 least likely, 1 most
vary = np.linspace(-.5, .5,num=2)
zin1 = np.vstack((vary,zeros,zeros))
zin2 = np.vstack((zeros,vary,zeros))
zin3 = np.vstack((zeros,zeros,vary))

zin1 = np.transpose(zin1)
zin1 = np.asarray(zin1)
zin1 = zin1.astype(float)
zin1 = torch.tensor(zin1)
zin1 = zin1.float()
zin1 = zin1.cuda()

zin2 = np.transpose(zin2)
zin2 = np.asarray(zin2)
zin2 = zin2.astype(float)
zin2 = torch.tensor(zin2)
zin2 = zin2.float()
zin2 = zin2.cuda()

zin3 = np.transpose(zin3)
zin3 = np.asarray(zin3)
zin3 = zin3.astype(float)
zin3 = torch.tensor(zin3)
zin3 = zin3.float()
zin3 = zin3.cuda()

dim = (2,3)
zin1sm = torch.reshape(zin1,dim)
zin2sm = torch.reshape(zin2,dim)
zin3sm = torch.reshape(zin3,dim)

with torch.no_grad():

    out1 = model.decode(zin1sm).view(-1, 1, 28, 28) 
    save_image(out1, os.path.join('varyl1mini.png'))
    out2 = model.decode(zin2sm).view(-1, 1, 28, 28)
    save_image(out2, os.path.join('varyl2mini.png'))
    out3 = model.decode(zin3sm).view(-1, 1, 28, 28) 
    save_image(out3, os.path.join('varyl3mini.png'))

2.4736133
1.5659255
2.1516063
